## Collecting Data

In [1]:
import numpy as np
import cv2
import sys
sys.path.insert(0, "/opt/intel/mkl/lib/intel64")
sys.path.insert(0, "/usr/local/lib/python3.6/site-packages")
import dlib
import numpy as np
import logging
import time

def dlibFacelandmarkToNumpy(shape, dtype="int"):
    coords = np.zeros((68, 2), dtype=dtype)
 
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)
 
    return coords
 
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

DATA_AMOUNT = 500



In [2]:
import pickle
import csv

STORE_LOCATION = 'C:\\Users\\Bone\\Desktop\\faces\\'

In [3]:

videoInput = cv2.VideoCapture(0)
data = np.ndarray((DATA_AMOUNT,68,2), dtype = float)
data_count = 0

while (data_count < DATA_AMOUNT):
    
    _,frame = videoInput.read()
   
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 
    rects = detector(gray, 0)
    for rect in rects:
#         print('found')
        shape = predictor(gray, rect)
        shape = dlibFacelandmarkToNumpy(shape)
        x, y, w, h = cv2.boundingRect(shape)
#         print(shape)
#         print('x:',x, 'y:', y, 'w:', w,'h:', h)
        shape[:,0] -= x
        shape[:,1] -= y
        shape_x = shape[:,0]
        shape_y = shape[:,1]
        shape = shape.astype(float)
        shape[:,0] = np.divide(shape_x, w)
        shape[:,1] = np.divide(shape_y, h)
        data[data_count,:,:] = shape
        print(int(data_count * 100 / DATA_AMOUNT),end = '\r')
#         print(shape)
        data_count += 1
# do a bit of cleanup
cv2.destroyAllWindows()
videoInput.release()
# print(data)
name = input("Enter name: ")
pickle.dump(data,open(STORE_LOCATION + name, 'wb'))
print('stored')



KeyboardInterrupt: 

## Get data

In [3]:
import torch
def unpickle(file):
    with open(file,'rb') as fo:
        output = pickle.load(fo, encoding = 'ASCII')
    return output
n = int(input('amount of people:'))

classes = list()

TRAINING_AMOUNT = int(n * DATA_AMOUNT * 4 / 5)
TEST_AMOUNT = int(n * DATA_AMOUNT * 1 / 5)

training_data = np.ndarray((TRAINING_AMOUNT,68*2))
training_labels = np.ndarray((TRAINING_AMOUNT,n))

test_data = np.ndarray((TEST_AMOUNT,68 * 2))
test_labels = np.ndarray((TEST_AMOUNT,n))

training_counter = 0
test_counter = 0



for i in range (n):
    name = input('name')
    classes.append(name)
    data = unpickle(STORE_LOCATION + name)
    
#     print(data)
    training_data[training_counter : training_counter + int(DATA_AMOUNT * 4 / 5),:] = data[: int(DATA_AMOUNT * 4 / 5), :, :].reshape(int(DATA_AMOUNT * 4 / 5), 68 * 2)
    test_data[test_counter : test_counter + int(DATA_AMOUNT * 1 / 5)] = data[int(DATA_AMOUNT * 4 / 5) :, :, :].reshape(int(DATA_AMOUNT * 1 / 5), 68 * 2)
   
    training_labels[training_counter :, :] = 0
    training_labels[training_counter : training_counter + int(DATA_AMOUNT * 4 / 5),i] = 1
    test_labels[test_counter :, :] = 0
    test_labels[test_counter : test_counter + int(DATA_AMOUNT * 1 / 5), i] = 1
    
    training_counter += int(DATA_AMOUNT * 4 / 5)
    test_counter += int(DATA_AMOUNT * 1 / 5)
training_data = torch.from_numpy(training_data)
training_labels = torch.from_numpy(training_labels)
# print(training_data[0])
# print(training_labels[0])
print(training_data.shape)
print(training_labels.shape)

print('done loading data')

amount of people:5
nameBone
nameChal
nameBook
nameP'An
nameP'Ton
torch.Size([2000, 136])
torch.Size([2000, 5])
done loading data


In [66]:
# import torch
# def unpickle(file):
#     with open(file,'rb') as fo:
#         output = pickle.load(fo, encoding = 'ASCII')
#     return output
# n = int(input('amount of people:'))

# classes = list()

# TRAINING_AMOUNT = int(n * DATA_AMOUNT * 4 / 5)
# TEST_AMOUNT = int(n * DATA_AMOUNT * 1 / 5)

# training_data = np.ndarray((TRAINING_AMOUNT,68*2))
# training_labels = np.ndarray((TRAINING_AMOUNT,1))

# test_data = np.ndarray((TEST_AMOUNT,68 * 2))
# test_labels = np.ndarray((TEST_AMOUNT,1))

# training_counter = 0
# test_counter = 0



# for i in range (n):
#     name = input('name')
#     classes.append(name)
#     data = unpickle(STORE_LOCATION + name)
    
# #     print(data)
#     training_data[training_counter : training_counter + int(DATA_AMOUNT * 4 / 5),:] = data[: int(DATA_AMOUNT * 4 / 5), :, :].reshape(int(DATA_AMOUNT * 4 / 5), 68 * 2)
#     test_data[test_counter : test_counter + int(DATA_AMOUNT * 1 / 5)] = data[int(DATA_AMOUNT * 4 / 5) :, :, :].reshape(int(DATA_AMOUNT * 1 / 5), 68 * 2)
   
#     training_labels[training_counter : training_counter + int(DATA_AMOUNT * 4 / 5)] = i
#     test_labels[test_counter : test_counter + int(DATA_AMOUNT * 1 / 5)] = i
    
#     training_counter += int(DATA_AMOUNT * 4 / 5)
#     test_counter += int(DATA_AMOUNT * 1 / 5)
# training_data = torch.from_numpy(training_data)
# training_labels = torch.from_numpy(training_labels)
# # print(training_data[0])
# # print(training_labels[0])
# print(training_data.shape)
# print(training_labels.shape)

# # print('done loading data')

amount of people:4
nameBone
nameChal
nameTanawinBook
namethipok
torch.Size([1600, 136])
torch.Size([1600, 1])


In [4]:
print(training_labels[1199])


 0
 0
 1
 0
 0
[torch.DoubleTensor of size 5]



In [5]:
from torch.utils.data import Dataset, DataLoader
class landmarksData(Dataset):
    def __init__(self, data, labels, classes,transform=None, data_dir=None):
        self.data = data
        self.labels = labels
        self.transform=transform
        self.classes = classes
    def __getitem__(self,index):
        single_landmark_labels = self.labels[index]
        single_landmarks = self.data[index]
        return (single_landmarks,single_landmark_labels)
    def __len__(self):
        return len(self.data)
training_set = landmarksData(training_data, training_labels, classes)
test_set = landmarksData(test_data, test_labels, classes)
print(training_set.__len__())
print(test_set.__len__())
# print(training_set.__getitem__(3))

2000
500


In [7]:
# print(training_data.type)
# print(training_labels.type)
# training_set = torch.utils.data.TensorDataset(training_data,training_labels)

In [6]:
trainloader = DataLoader(training_set, batch_size=4,shuffle=True)
dataiter = iter(trainloader)
landmarks, label = dataiter.next()
print("landmarks: ",landmarks[0])
print("label: ",label[0])
testloader = DataLoader(test_set, batch_size=4,shuffle=True)
testiter = iter(testloader)


landmarks:  
 0.0000
 0.3130
 0.0168
 0.4431
 0.0462
 0.5650
 0.0798
 0.6870
 0.1345
 0.7967
 0.2227
 0.8862
 0.3361
 0.9512
 0.4580
 0.9919
 0.5840
 0.9959
 0.7059
 0.9675
 0.8025
 0.8943
 0.8866
 0.8049
 0.9496
 0.6992
 0.9832
 0.5813
 0.9958
 0.4553
 0.9958
 0.3252
 0.9832
 0.1951
 0.0588
 0.1951
 0.1092
 0.1098
 0.2017
 0.0650
 0.3109
 0.0610
 0.4160
 0.0894
 0.5294
 0.0691
 0.6261
 0.0244
 0.7311
 0.0000
 0.8361
 0.0163
 0.9118
 0.0854
 0.4874
 0.1992
 0.4958
 0.2805
 0.5042
 0.3618
 0.5126
 0.4472
 0.3992
 0.5407
 0.4622
 0.5407
 0.5210
 0.5447
 0.5798
 0.5325
 0.6387
 0.5203
 0.1639
 0.2683
 0.2185
 0.2398
 0.2815
 0.2317
 0.3487
 0.2561
 0.2857
 0.2683
 0.2227
 0.2724
 0.6345
 0.2236
 0.6849
 0.1870
 0.7521
 0.1829
 0.8151
 0.1951
 0.7605
 0.2154
 0.6975
 0.2236
 0.3361
 0.7439
 0.3992
 0.6829
 0.4706
 0.6423
 0.5294
 0.6504
 0.5840
 0.6301
 0.6639
 0.6585
 0.7395
 0.7073
 0.6765
 0.7805
 0.6092
 0.8171
 0.5462
 0.8293
 0.4832
 0.8293
 0.4118
 0.8049
 0.3739
 0.7398
 0.4748
 0.

# Training

In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
def foo(l, dtype=float):
    return map(dtype, l)

In [8]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(68 * 2, 68 * 2)
        self.fc2 = nn.Linear(68 * 2, 64)
        self.fc3 = nn.Linear(64, n)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()
# net.cuda()
print(net)

Net (
  (fc1): Linear (136 -> 136)
  (fc2): Linear (136 -> 64)
  (fc3): Linear (64 -> 5)
)


In [9]:
import torch.optim as optim

criterion =  nn.MSELoss()
# optimizer = optim.SGD(net.parameters(), lr = 0.0001, momentum = 0.9)
optimizer = optim.SGD(net.parameters(), lr = 0.0001)

In [12]:
for epoch in range (10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs.float()), Variable(labels.float())
        if(i == 1):
            labelss=labels[:,0]
#             print("inputs: ", inputs)
            print("outputs: ", outputs)
            print("labels:", labelss)
        optimizer.zero_grad()
        outputs = net(inputs)
#         print('outputs:',outputs)
#         print(labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.data[0]
        if(i % 100 == 99):
            print('[%d, %5d] loss: %.3[[1. 0.]]f' %
                (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
print('finished training')

outputs:  Variable containing:
1.00000e-02 *
 -0.0386  1.9010  5.6941  6.7038  3.0685
  0.3567  1.5441  5.7465  7.0456  2.9810
  0.3551  1.1564  5.8447  7.1089  3.3287
  0.3838  1.3767  5.8212  7.0896  3.2119
[torch.FloatTensor of size 4x5]

labels: Variable containing:
 0
 0
 0
 0
[torch.FloatTensor of size 4]

[1,   100] loss: 0.187
[1,   200] loss: 0.186
[1,   300] loss: 0.185
[1,   400] loss: 0.185
[1,   500] loss: 0.184
outputs:  Variable containing:
1.00000e-02 *
  1.7352  2.0731  6.7017  8.2924  4.3827
  1.6622  2.3206  6.6404  8.0705  4.0067
  1.4405  2.6710  6.3677  8.1623  4.1760
  1.8780  2.3211  7.0002  8.1625  3.9602
[torch.FloatTensor of size 4x5]

labels: Variable containing:
 0
 0
 1
 1
[torch.FloatTensor of size 4]

[2,   100] loss: 0.183
[2,   200] loss: 0.183
[2,   300] loss: 0.181
[2,   400] loss: 0.183
[2,   500] loss: 0.181
outputs:  Variable containing:
1.00000e-02 *
  2.8552  3.0782  7.4065  9.3539  5.4909
  3.2303  3.2296  8.0828  9.4730  5.6094
  2.9211  3.311

# Test accuracy

In [23]:
def outputToTensor(labels):
    # print(labels)
    # print(labels.numpy().astype(int))
    tmp = labels.numpy().astype(int)
    # print([ np.where(r==1)[0][0] for r in tmp ])
    tmp = [ np.where(r==1)[0][0] for r in tmp ]
    tmp = np.array(tmp)
    tmp = torch.from_numpy(tmp)
    # print(tmp)
    return tmp

In [37]:
dataiter  = iter(testloader)
landmarks,labels = dataiter.next()
print(labels)
tmp = outputToTensor(labels)
print('GroundTruth:' ,' '.join('%5s'% classes[int(tmp[j])] for j in range(4)))

outputs = net(Variable(landmarks.float()))
print('outputs:',outputs)
_,predicted = torch.max(outputs.data, 1)
print(predicted)
print('Predicted :',' '.join('%5s'% classes[int(predicted[j])] for j in range(4) ))


    0     0     0     0     0     0     0     1     0     0
    0     0     0     0     0     0     0     0     0     1
    0     0     0     0     0     0     0     1     0     0
    0     0     1     0     0     0     0     0     0     0
[torch.DoubleTensor of size 4x10]

GroundTruth:   Tan   Mum   Tan  Book
outputs: Variable containing:
-0.0790 -0.0196  0.3560  0.4597  0.0458 -0.0823  0.1725  0.2134 -0.0267 -0.0705
 0.1431  0.0125  0.0211  0.0604 -0.0033 -0.0352 -0.0003  0.0957  0.2255  0.4799
-0.0761 -0.0170  0.2782  0.3745  0.0427 -0.0733  0.1284  0.2982  0.0667 -0.0408
 0.5559  0.0796  0.0879  0.0069 -0.0152  0.1711  0.0655  0.0926 -0.0749 -0.0059
[torch.FloatTensor of size 4x10]


 3
 9
 3
 0
[torch.LongTensor of size 4]

Predicted : Thipok   Mum Thipok  Bone


In [25]:
correct = 0
total = 0
i = 0
for data in testloader:
    i += 1
    images,labels = data
    outputs = net(Variable(images.float()))
#     if( i < 5): 
#         print(labels)
#         print('output:',torch.max(outputs.data,1))
#     print(outputs)
    _,predicted = torch.max(outputs.data, 1)
#     print('predicted: ',predicted)
#     print('labels: ',labels)
    labels = outputToTensor(labels)
    total += labels.size(0)
    correct += (predicted == labels).sum()    
print('Accuracy :',(correct/total*100))

Accuracy : 70.39999999999999


In [26]:
print(net)

Net (
  (fc1): Linear (136 -> 136)
  (fc2): Linear (136 -> 64)
  (fc3): Linear (64 -> 10)
)


# Saving model

In [56]:
name = input('name: ')
torch.save(net.state_dict(), 'C:\\Users\\Bone\\Desktop\\faces\\models\\'+name)
# print('a')

name: secondTry


In [27]:
name = input('name: ')
pickle.dump(net,open('C:\\Users\\Bone\\Desktop\\faces\\models\\' + name, 'wb'))
print('saved')

name: 10 person
saved
